In [1]:
import plotly.express as px
import pandas as pd
import json
import os
os.environ['USER'] = "rmoine"
from pathlib import Path
import numpy as np
from typing import *
from llama.main import compute_metrics_from_files, DatasetName, compute_metrics_from_list, get_tokenizer
import llama.main as m
from sklearn.metrics import f1_score, auc, roc_curve
from PIL import Image
from pandasql import sqldf
from io import BytesIO
import base64
from plotly.graph_objects import Figure
import win32clipboard
import textwrap
import colorsys
import optuna
import tqdm
import re
def img_to_clipboard(fig: Figure):
    img_bytes = fig.to_image(format="png")
    image = Image.open(BytesIO(img_bytes))
    
    output = BytesIO()
    image.convert('RGB').save(output, 'BMP')
    data = output.getvalue()[14:]
    output.close()
    win32clipboard.OpenClipboard()
    win32clipboard.EmptyClipboard()
    win32clipboard.SetClipboardData(win32clipboard.CF_DIB, data)
    win32clipboard.CloseClipboard()
root = Path(f"../../data/baseline_studies/")

c:\Users\robin\miniconda3\envs\severityPrediction\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
Import of import evaluate failed
Import of from pytorchtools import EarlyStopping failed


In [3]:
l = []
for i,f in enumerate(root.rglob("*.json")):
    algorithm_group: Literal['bayesian_network','svc','knn']
    if "_nb_" in f.stem:
        algorithm_group = 'bayesian_network'
    elif "_svm_knn_" in f.stem:
        if "KNeighborsClassifier" in f.stem:
            algorithm_group = "knn"
        elif "SVC_" in f.stem:
            algorithm_group = "svc"
        else:
            raise Exception
    else:
        raise Exception
    num_samples = int(re.search("([0-9]+)_samples", f.stem).group(1))
            
    with open(f) as fp:
        data = json.load(fp)[0]
    dataset_choice = "eclipse_72k" if "eclipse_72k" in f.stem else "mozilla_200k"
    fixed_params = {k:v for k,v in data.items() if not isinstance(v, list)}
    fixed_params["params"] = fixed_params
    fixed_params["run_id"] = i
    fixed_params["num_samples"] = num_samples
    fixed_params["algorithm_group"] = algorithm_group
    fixed_params["dataset_choice"] = dataset_choice
    infos = {**fixed_params}
    for dataset in ["val","test"]:
        d = [
            {"binary_severity":b, "severity_pred": p, "bug_id": i} 
            for i,(b,p) in enumerate(zip(
                data[f'binary_severity_{dataset}'],
                data[f'y_{dataset}_pred']
                ))]
        threasholds = sorted(np.unique(filter(lambda x:x is not None,data[f'thresholds_{dataset}'])))
        t = 0.5
        confusion_matrix, f1, _ = compute_metrics_from_list(d, pred_field="severity_pred")
        accuracy = np.sum(np.diag(confusion_matrix)) / np.sum(confusion_matrix)
        infos[f"thresholdtest_accuracy_{dataset}"] = accuracy
        infos[f"thresholdtest_f1_{dataset}"] = f1
        infos[f"threashold_used_{dataset}"] = t
    l.append(infos)
df = pd.DataFrame(l)

In [4]:
d = df.query("dataset_choice == 'eclipse_72k'")
d.sort_values("num_samples",inplace=True)
px.line(d,x="num_samples",y="thresholdtest_accuracy_val",color="algorithm_group").show()
d.sort_values("thresholdtest_accuracy_val",inplace=True,ascending=False)
display(d.head()[["auc_val","thresholdtest_accuracy_val","classifier"]])

C:\Users\robin\AppData\Local\Temp\ipykernel_45460\1383751435.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d.sort_values("num_samples",inplace=True)


C:\Users\robin\AppData\Local\Temp\ipykernel_45460\1383751435.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,auc_val,thresholdtest_accuracy_val,classifier
9,0.654523,0.705686,KNeighborsClassifier
11,0.654523,0.705686,KNeighborsClassifier
13,0.654523,0.705686,KNeighborsClassifier
8,0.641985,0.692308,KNeighborsClassifier
12,0.589221,0.622074,KNeighborsClassifier


In [43]:
d = df.query("dataset_choice == 'mozilla_200k'")
d.sort_values("num_samples",inplace=True)
px.line(d,x="num_samples",y="thresholdtest_accuracy_val",color="algorithm_group").show()
d.sort_values("thresholdtest_accuracy_val",inplace=True,ascending=False)
display(d.head()[["auc_val","thresholdtest_accuracy_val","classifier"]])

C:\Users\robin\AppData\Local\Temp\ipykernel_18588\1028182460.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



C:\Users\robin\AppData\Local\Temp\ipykernel_18588\1028182460.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,auc_val,thresholdtest_accuracy_val,classifier
14,0.562418,0.729231,KNeighborsClassifier
16,0.601532,0.607832,KNeighborsClassifier
6,0.710356,0.384804,GaussianNB
7,0.634373,0.379111,GaussianNB
5,0.660247,0.317125,GaussianNB


In [27]:
d = df.query("dataset_choice == 'eclipse_72k'").query("auc_val == auc_val.max()").query("thresholdtest_accuracy_val == thresholdtest_accuracy_val.max()")
display(d.iloc[0])
d = df.query("dataset_choice == 'mozilla_200k'").query("auc_val == auc_val.max()").query("thresholdtest_accuracy_val == thresholdtest_accuracy_val.max()")
display(d.iloc[0])

seed                                                                           0
classifier                                                                   SVC
train_fun                                                                  train
train_accuracy                                                          0.843209
fold_id                                                                        0
auc_val                                                                 0.756181
classifier_name                                                              SVC
alpha                                                                        NaN
fit_prior                                                                    NaN
binarize                                                                     NaN
auc_test                                                                0.756181
params                         {'seed': 0, 'classifier': 'SVC', 'train_fun': ...
run_id                      

seed                                                                           0
classifier                                                            GaussianNB
train_fun                                                                  train
train_accuracy                                                             0.786
fold_id                                                                        0
auc_val                                                                 0.710356
classifier_name                                                       GaussianNB
alpha                                                                        NaN
fit_prior                                                                    NaN
binarize                                                                     NaN
auc_test                                                                0.710356
params                         {'seed': 0, 'classifier': 'GaussianNB', 'train...
run_id                      